In [1]:
import psycopg2 as ps
import pandas as pd
import tokenkey as tk

In [2]:
df = pd.read_csv('D:\Program File\Python\AllEnv\envlatihan\Latihan\RequestAPI\Youtube\youtube_pull.csv')
print(df.head())

      video_id                                        video_title upload_date  \
0  30hS-MjpU6E  SQL Syntax Best Practices: How to Structure Yo...  2021-12-22   
1  Bgpp99iz0I0  Most Common Coding Mistakes on Data Science In...  2021-11-04   
2  GeJUvdkJKEc  Multiple Solutions to Data Scientist Interview...  2021-10-11   
3  GGURenNfXI0  Multiple Approaches for Solving a Twitter Data...  2021-09-13   
4  PlpUo6bHsBQ  Advanced Facebook Data Science SQL interview q...  2021-08-09   

   view_count  like_count  dislike_count  comment_count  
0        2978         182            NaN             18  
1        3108         153            NaN             27  
2        4151         126            NaN             32  
3        3229         144            NaN             32  
4       10621         341            NaN             45  


In [3]:
def connect_to_db(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

conn = connect_to_db(tk.HOST_NAME_Postgres, tk.DBNAME_Postgres, tk.USERNAME_Postgres, tk.PASSWORD_Postgres, tk.PORT_Postgres)
curr = conn.cursor()

Connected!


In [4]:
sql = """DROP TABLE videos"""
curr.execute(sql)
conn.commit()

In [5]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

create_table(curr)
conn.commit()

In [6]:
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM videos WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['comment_count'])


# Buat liat misal datanya udah ada dia cuma bakal update doang datanya berdasarkan idnya, klo belom ada di append ke temp_df dulu
# Nanti temp_df bakal di append ke db lewat func append_from_df_to_db()
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [7]:
#update data for existing videos

new_vid_df = update_db(curr,df)
conn.commit()

In [8]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()